### ライブラリのインポート

In [1]:
import requests                  
from bs4 import BeautifulSoup    
import re                        
import pandas as pd             

### HTMLから欲しいテキストを抽出する

In [2]:
urls = [
    "https://magi.camp/blogs/167305889",
    "https://magi.camp/blogs/47058427", 
    "https://magi.camp/blogs/774748184"   
]

# 各ページごとに、lines（テキスト行）と titles（パック名）を抽出
all_lines = []
all_titles = []

for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # 全体のテキストを1行ごとに分割
    lines = soup.get_text(separator="\n").split("\n")
    all_lines.append(lines)

    # タイトル（パック名）は <p>タグから抽出（「拡張パック」を含むものすべて対象）
    titles = []
    for p in soup.find_all("p"):
         text = p.get_text(strip=True)
         if "拡張パック" in text:
              titles.append(text.replace("■", "").strip())
 
    all_titles.append(titles)

### 抽出したテキストから、「パック名、発売日、希望小売価格、カード枚数」の情報に切り分ける

In [3]:
data = []

# 各ページごとに処理
for page_index in range(len(all_lines)):
    lines = all_lines[page_index]
    titles = all_titles[page_index]
    title_index = 0
    i = 0

    while i < len(lines):
        line = lines[i].strip()

        if "発売日" in line:
            release = re.search(r"発売日[:：]?\s*(\d{4}年\d{1,2}月\d{1,2}日)", line)
            price   = re.search(r"希望小売価格[:：]?\s*([0-9,]+円)", lines[i+1].strip()) if i+1 < len(lines) else None
            count   = re.search(r"カード[／/]\s*([0-9]+)枚", lines[i+2].strip()) if i+2 < len(lines) else None

            if release and price and count:
                title = titles[title_index] if title_index < len(titles) else "不明"
                data.append({
                    "パック名": title,
                    "発売日": release.group(1),
                    "希望小売価格": price.group(1),
                    "カード枚数": count.group(1)
                })
                title_index += 1
                i += 3
            else:
                i += 1
        else:
            i += 1

### 切り分けたデータをデータフレームにまとめて、CSVで出力する

In [4]:
df = pd.DataFrame(data)
df.to_csv("pokemon_cards_with_titles.csv", index=False, encoding="utf-8-sig")
df.head()

,パック名,発売日,希望小売価格,カード枚数
0,第1弾 拡張パック,1996年10月20日,300円,10
1,第2弾 拡張パック『ポケモンジャングル』,1997年3月5日,300円,10
2,第3弾 拡張パック『化石の秘密』,1997年6月21日,300円,10
3,第4弾 拡張パック『ロケット団』,1997年11月21日,300円,10
4,拡張パック 第1弾『金、銀、新世界へ...』,1998年10月24日,300円,10
